# Ejercicios<a class="tocSkip">
## Ciencia de Datos <a class="tocSkip">
### Ingenieria Electrónica <a class="tocSkip">
### Universidad Popular del Cesar <a class="tocSkip">
### Prof.: Jose Ramón Iglesias Gamarra - [https://github.com/joseramoniglesias/](https://github.com/joseramoniglesias/) <a class="tocSkip">
  **joseiglesias@unicesar.edu.co**

# Cómo extraer información de PDFs mediante Python


<img src="https://i.ibb.co/Rg97YGX/pdf.png" style="width:300px;">



---------------

## Caso de uso:
Veremos cómo extraer información de los PDFs reportados por un Fondo de Inversión a la CNMV para facilitar análisis sobre esos datos.

http://www.cnmv.es/Portal/Consultas/IIC/Fondo.aspx?nif=V87758306

## Requisitos:
- pymupdf
- pandas
- re
---------------------------

In [ ]:
!pip install pymupdf
!pip install PyPDF2
!pip install frontend

In [19]:
import fitz
import pymupdf
import re
import pandas as pd

Después de cargar las librerías que vamos a utilizar leeremos el archivo PDF página por página y almacenaremos el contenido en la variable “text”

Adicionalmente reemplazamos los símbolos � que son errores a la hora de decodificar el PDF:

In [20]:
with pymupdf.open("Q1.pdf") as doc:
    text = ""
    for page in doc:
        text += page.get_textbox(page)
text = text.replace('�','')
print(text)

 1
COBAS INTERNACIONAL, FI
Nº Registro CNMV: 5130
 
Informe Semestral del Segundo Semestre 2023 
 
Gestora: COBAS ASSET MANAGEMENT, SGIIC, S.A.        Depositario: BANCO INVERSIS, S.A.        Auditor:
DELOITTE, S.L.
Grupo Gestora: SANTA COMBA GESTIÓN        Grupo Depositario: BANCA MARCH        Rating Depositario: ND 
 
 
El presente informe, junto con los últimos informes periódicos, se encuentran disponibles por medios telemáticos
en www.cobasam.com.
 
La Entidad Gestora atenderá las consultas de los clientes, relacionadas con las IIC gestionadas en: 
Dirección
Paseo de la Castellana, 53, 2º
28046 - Madrid
917556800
 
Correo Electrónico
info@cobasam.com
 
Asimismo cuenta con un departamento o servicio de atención al cliente encargado de resolver las quejas y
reclamaciones. La CNMV también pone a su disposición la Oficina de Atención al Inversor (902 149 200, e-mail:
inversores@cnmv.es).
INFORMACIÓN FONDO
Fecha de registro: 03/03/2017
 
1. Política de inversión y divisa de denominació

Una vez hemos conseguido almacenar todo el archivo PDF en nuestra variable “text”, tendremos que separar cada una de las filas en una lista.

Para ello haremos uso de la función “split” para separar el texto cuando encuentre un salto de línea (\n):

In [21]:
lines = text.split('\n')

In [22]:
print(lines)

[' 1', 'COBAS INTERNACIONAL, FI', 'Nº Registro CNMV: 5130', ' ', 'Informe Semestral del Segundo Semestre 2023 ', ' ', 'Gestora: COBAS ASSET MANAGEMENT, SGIIC, S.A.        Depositario: BANCO INVERSIS, S.A.        Auditor:', 'DELOITTE, S.L.', 'Grupo Gestora: SANTA COMBA GESTIÓN        Grupo Depositario: BANCA MARCH        Rating Depositario: ND ', ' ', ' ', 'El presente informe, junto con los últimos informes periódicos, se encuentran disponibles por medios telemáticos', 'en www.cobasam.com.', ' ', 'La Entidad Gestora atenderá las consultas de los clientes, relacionadas con las IIC gestionadas en: ', 'Dirección', 'Paseo de la Castellana, 53, 2º', '28046 - Madrid', '917556800', ' ', 'Correo Electrónico', 'info@cobasam.com', ' ', 'Asimismo cuenta con un departamento o servicio de atención al cliente encargado de resolver las quejas y', 'reclamaciones. La CNMV también pone a su disposición la Oficina de Atención al Inversor (902 149 200, e-mail:', 'inversores@cnmv.es).', 'INFORMACIÓN FONDO'

In [23]:
len(lines)

2139

Tal y como vemos, después de utilizar el método “split” tendremos cada línea separada por comas en variable la “lines”.

Del total de las 2139 líneas, solo nos interesan aquellas que contienen información relativa a acciones de empresas cotizadas.

Con el fin de identificar aquellas líneas que contienen códigos ISIN, haremos uso de una expresión regular (REGEX)

Quizás este paso es el más complicado ya que vamos a utilizar expresiones regular y list comprehension.

Básicamente vamos a:

Iterar por cada elemento de la variable “lines”: pondremos un 1 si encuentra un ISIN con REGEX y un 0 si no encuentra ningún ISIN
Posteriormente obtendremos los índices para todas aquellas posiciones donde existe un ISIN
Utilizaremos la expresión ^\w{2}\d{9} que básicamente significa: encuentra un texto que cumpla el formato ISIN (2 letras + 9 dígitos)

In [24]:
index_rv = [1 if re.match(r'^\w{2}\d{10}', line) else 0 for line in lines]

Y ahora nos quedamos solo con los índices (posición en la lista index_rv) para aquellos valores que tienen 1 en index_rv:

In [25]:
index_rv = [i for i, s in enumerate(index_rv) if s==1 in index_rv]

In [26]:
index_rv

[1859,
 1865,
 1883,
 1907,
 1913,
 1919,
 1937,
 1949,
 1955,
 1961,
 1967,
 1979,
 1985,
 1997,
 2003,
 2015,
 2027]

Ahora podemos seleccionar uno de esos índices en la variable “lines” y ver qué obtenemos:

In [28]:
lines[1859]

'IL0010827181 - ACCIONES|Taro Pharmaceutical'

In [29]:
lines[1859:1859+6]

['IL0010827181 - ACCIONES|Taro Pharmaceutical',
 'USD',
 '0',
 '0,00',
 '1.792',
 '0,32']

Así mismo, si nos fijamos, después de cada ISIN + nombre de la acción tenemos 5 valores que corresponden a las columnas del PDF:

ISIN + Nombre acción
Divisa
Valor de Mercado Actual
% Actual del fondo
Valor de Mercado Pasado
% Pasado del fondo
Como los datos de cada fila van en bloques de 6 en 6, iteraremos por cada uno de los índices almacenados en index_rv y simplemente almacenaremos el resultado en una lista de listas llamada result:

Nota: en este caso he aprovechado para separar el ISIN del nombre y eliminar el texto “ACCIONES |” ya que no aporta ninguna información adicional

In [30]:
result = []
for i in index_rv:
    stock = lines[i:i+6]
    stock = stock[0].replace('ACCIONES|','').split(' - ') + stock[1:]
    result.append([p.strip() for p in stock])
    
pd.DataFrame(result, columns = ['ISIN','Nombre', 'Divisa', 'Actual_VM', 'Actual_%', 'Pasado_VM', 'Pasado_%'])

,ISIN,Nombre,Divisa,Actual_VM,Actual_%,Pasado_VM,Pasado_%
0,IL0010827181,Taro Pharmaceutical,USD,0,"0,00",1.792,"0,32"
1,SE0007897079,Academedia AB,SEK,13.568,"2,42",8.741,"1,57"
2,IT0005105231,Maire Tecnimont SPA,EUR,10.642,"1,90",9.442,"1,70"
3,IT0005241762,Cofide,EUR,21.329,"3,81",18.753,"3,37"
4,FR0013181864,CGG,EUR,0,"0,00",6.129,"1,10"
5,DE0005785604,Fresenius SE,EUR,8.874,"1,58",9.026,"1,62"
6,US0082521081,Affil Managers,USD,8.159,"1,46",8.751,"1,57"
7,US5006881065,Kosmos Energy LTD,USD,8.561,"1,53",9.584,"1,72"
8,KR7028260008,Samsung C&T Corp,KRW,10.637,"1,90",10.183,"1,83"
9,IT0000076486,DanieCo,EUR,20.804,"3,72",19.248,"3,46"


**Copyright**

The notebooks are provided as [Open Educational Resource](https://de.wikipedia.org/wiki/Open_Educational_Resources). Feel free to use the notebooks for your own educational purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).